<a href="https://colab.research.google.com/github/Data-FCDP/Forsyth-County-NC-Voter-Party-Movement-Analysis/blob/main/marshallj_(Jul_24%2C_2025%2C_10_31_40%E2%80%AFPM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2025 County Convention Attend Match
Creates a new table in the demsncforsythcp.commons dataset

*   Create a new tablein the demsncforsythcp.commons dataset
*   Table name: 2025_County_Convention_Attend_2
*   AS means we're creating the table and populating it with data from a SELECT query






In [ ]:
CREATE TABLE `demsncforsythcp.commons.2025_County_Convention_Attend_2` AS

# Required Voter File Match Columns
matchback_id: Required by the voter file match system - uses the person's VAN

*   Required Voter File Match Columns
matchback_id: Required by the voter file match system - uses the person's VAN ID as unique
*   state_code: Required field set to 'NC' (North Carolina) for all records




In [ ]:
es.myc_van_id as matchback_id,
'NC' as state_code,

# Person Id

*   Keeps both the event signup VAN ID and person record VAN ID for reference
*   These are different ID systems that link the same person



In [ ]:
es.myc_van_id,
pr.myv_van_id,

# Name Information

*   Gets the person's full name from the person records table
*   Essential for identifying convention attendees



In [ ]:
pr.first_name,
pr.middle_name,
pr.last_name,

# Personal Details

*   Includes birth date for demographic analysis and verification



In [ ]:
pr.date_of_birth,

# Address Information

*   Gets current address from the addresses table
*   Used for geographic analysis and mailings



In [ ]:
addr.address,
addr.city,
addr.zip,

# Event Details

*   Captures details about the specific event (453086) they signed up for
*   Tracks when the event occurred and their signup status


In [ ]:
es.event_signup_id,
es.event_id,
es.datetime_offset_begin as event_datetime,
es.datetime_offset_end as event_end_datetime,
es.current_event_signups_event_status_id,

# Convention Tracking Fields

*   Creates placeholder fields for future convention attendance tracking
*   LAll start as NULL and can be updated later when actual attendance is confirmed



In [ ]:
CAST(NULL AS TIMESTAMP) as convention_attended_datetime,
CAST(NULL AS STRING) as convention_attendance_status,
CAST(NULL AS BOOLEAN) as attended_county_convention,

# Metadata

*   Tracks when this record was created and where the data came from
*   Helps with data lineage and debugging



In [ ]:
CURRENT_TIMESTAMP() as record_created,
'Event_453086_Signup' as source,

# FROM and JOIN Clauses

*   FROM: Starts with event signups as the main table
*   JOIN: Links to person records to get names and details (INNER JOIN - only people with matching records)
*   LEFT JOIN: Links to addresses (LEFT JOIN - keeps people even if no address found)
*   datetime_suppressed IS NULL: Only gets current/active addresses





In [ ]:
FROM `demsncforsythcp.vansync.event_signups` es
JOIN `demsncsp.vansync.person_records_myc` pr
  ON es.myc_van_id = pr.myc_van_id
LEFT JOIN `demsncforsythcp.vansync.contacts_addresses_myc` addr
  ON es.myc_van_id = addr.myc_van_id
  AND addr.datetime_suppressed IS NULL

# WHERE Clause

*   Only includes signups for the specific event (453086)
*   Excludes cancelled signups (where datetime_suppressed is not null)



In [ ]:
WHERE es.event_id = '453086'
  AND es.datetime_suppressed IS NULL;